# Случайные леса

В этом задании вам предстоит реализовать ансамбль деревьев решений, известный как случайный лес, применить его к публичным данным пользователей социальной сети Вконтакте, и сравнить его эффективность с ансамблем, предоставляемым библиотекой CatBoost.

В результате мы сможем определить, какие подписки пользователей больше всего влияют на определение возраста и пола человека. 

In [1]:
from sklearn.model_selection import train_test_split
import numpy as np
import pandas
import random
import matplotlib.pyplot as plt
import matplotlib
import copy
from catboost import CatBoostClassifier

In [2]:
from typing import Union, List, Dict, Any
from copy import deepcopy
from collections import Counter

In [3]:
def gini(x):
    _, counts = np.unique(x, return_counts=True)
    proba = counts / len(x)
    return np.sum(proba * (1 - proba))
    
def entropy(x):
    _, counts = np.unique(x, return_counts=True)
    proba = counts / len(x)
    return -np.sum(proba * np.log2(proba))

def gain(left_y, right_y, criterion):
    y = np.concatenate((left_y, right_y))
    return criterion(y) - (criterion(left_y) * len(left_y) + criterion(right_y) * len(right_y)) / len(y)

### Задание 1 (2 балла)
Random Forest состоит из деревьев решений. Каждое такое дерево строится на одной из выборок, полученных при помощи bagging. Элементы, которые не вошли в новую обучающую выборку, образуют out-of-bag выборку. Кроме того, в каждом узле дерева мы случайным образом выбираем набор из `max_features` и ищем признак для предиката разбиения только в этом наборе.

Сегодня мы будем работать только с бинарными признаками, поэтому нет необходимости выбирать значение признака для разбиения.

#### Методы
`predict(X)` - возвращает предсказанные метки для элементов выборки `X`

#### Параметры конструктора
`X, y` - обучающая выборка и соответствующие ей метки классов. Из нее нужно получить выборку для построения дерева при помощи bagging. Out-of-bag выборку нужно запомнить, она понадобится потом.

`criterion="gini"` - задает критерий, который будет использоваться при построении дерева. Возможные значения: `"gini"`, `"entropy"`.

`max_depth=None` - ограничение глубины дерева. Если `None` - глубина не ограничена

`min_samples_leaf=1` - минимальное количество элементов в каждом листе дерева.

`max_features="auto"` - количество признаков, которые могут использоваться в узле. Если `"auto"` - равно `sqrt(X.shape[1])`

In [4]:
class DecisionTreeLeaf:
    """

    Attributes
    ----------
    y : Тип метки (напр., int или str)
        Метка класса, который встречается чаще всего среди элементов листа дерева
    """
    def __init__(self, ys, classes):
        keys, counts = np.unique(ys, return_counts=True)
        self.probs = dict(zip(classes, np.zeros(len(classes))))
        for key, cnt in zip(keys, counts):
            self.probs[key] = cnt / len(ys)
        self.y = keys[np.argmax(counts)]

class DecisionTreeNode:
    """

    Attributes
    ----------
    split_dim : int
        Измерение, по которому разбиваем выборку.
    split_value : float
        Значение, по которому разбираем выборку.
    left : Union[DecisionTreeNode, DecisionTreeLeaf]
        Поддерево, отвечающее за случай x[split_dim] < split_value.
    right : Union[DecisionTreeNode, DecisionTreeLeaf]
        Поддерево, отвечающее за случай x[split_dim] >= split_value. 
    """
    def __init__(self, split_dim: int, 
                 left: Union['DecisionTreeNode', DecisionTreeLeaf], 
                 right: Union['DecisionTreeNode', DecisionTreeLeaf]):
        self.split_dim = split_dim
        self.left = left
        self.right = right

In [5]:
class DecisionTree:
    def __init__(self, criterion="gini", max_depth=None, min_samples_leaf=1, max_features="auto"):
        self.criterion = gini if criterion == 'gini' else entropy
        self.max_depth = max_depth
        self.min_samples_leaf = min_samples_leaf
        self.max_features = np.sqrt(X.shape[1]) if max_features == 'auto' else max_features
        
    def fit(self, X, y):    
        self.classes = np.unique(y)
        self.root = self._build_(X, y, np.arange(len(y)))
        
    def predict(self, X):
        raise NotImplementedError()
        
    def _build_(self, X: np.ndarray, y: np.ndarray, indices: np.ndarray, depth=0) -> Union[DecisionTreeNode, DecisionTreeLeaf]: 
        if self.max_depth is not None and depth >= self.max_depth:
            return DecisionTreeLeaf(y[indices], self.classes)
                        
        n_samples, n_features = len(indices), X.shape[1]
        picked_features = np.random.choice(n_features, int(self.max_features), replace=False)
        
        best_ig = 0
        split_dim, split_pos = None, None
        for dim in picked_features:
            sorted_indices = (np.array(sorted(np.vstack((X[indices, dim], indices)).T, key=lambda x: x[0]))[:,1]).astype(int)
                
            i = np.argmax(X[sorted_indices, dim])
            ig = gain(y[sorted_indices[:(i + 1)]], y[sorted_indices[(i + 1):]], self.criterion)
                
            if ig > best_ig and min(i + 1, n_samples - i - 1) >= self.min_samples_leaf:
                split_dim = dim
                split_pos = i + 1
                best_ig = ig
            
        if best_ig > 0:
            sorted_indices = (np.array(sorted(np.vstack((X[indices, split_dim], indices)).T, key=lambda x: x[0]))[:,1]).astype(int)
            
            return DecisionTreeNode(split_dim=split_dim,
                                    left=self._build_(X, y, sorted_indices[:split_pos], depth + 1), 
                                    right=self._build_(X, y, sorted_indices[split_pos:], depth + 1))
        else:
            return DecisionTreeLeaf(y[indices], self.classes)
        
    def predict_proba(self, X: np.ndarray):
        
        ans = []
        for x in X:
            node = self.root
            
            while isinstance(node, DecisionTreeNode):
                if x[node.split_dim] == 0:
                    node = node.left
                else:
                    node = node.right
                    
            ans.append(node.probs)
        return ans
    
    def predict(self, X : np.ndarray):
        
        proba = self.predict_proba(X)
        return [max(p.keys(), key=lambda k: p[k]) for p in proba]
    

### Задание 2 (2 балла)
Теперь реализуем сам Random Forest. Идея очень простая: строим `n` деревьев, а затем берем модальное предсказание.

#### Параметры конструктора
`n_estimators` - количество используемых для предсказания деревьев.

Остальное - параметры деревьев.

#### Методы
`fit(X, y)` - строит `n_estimators` деревьев по выборке `X`.

`predict(X)` - для каждого элемента выборки `X` возвращает самый частый класс, который предсказывают для него деревья.

In [6]:
class RandomForestClassifier:
    def __init__(self, criterion="gini", max_depth=None, min_samples_leaf=1, max_features="auto", n_estimators=10):
        self.trees = []
        for _ in range(n_estimators):
            self.trees.append(DecisionTree(criterion, max_depth, min_samples_leaf, max_features))
    
    def fit(self, X, y):
        self.oobs = []
        
        for tree in self.trees:
            A = np.arange(X.shape[0])
            train_indices = np.random.choice(A, X.shape[0], replace=True)
            oobs_indices = A[~np.isin(A, train_indices)]
            
            self.oobs.append([X[oobs_indices], y[oobs_indices]])
            
            tree.fit(X[train_indices], y[train_indices])
    
    def predict(self, X): # soft
        ranks = np.sum([list(map(lambda d: list(d.values()), tree.predict_proba(X))) for tree in self.trees], axis=0)
        return self.trees[0].classes[np.argmax(ranks, axis=1)]

### Задание 3 (2 балла)
Часто хочется понимать, насколько большую роль играет тот или иной признак для предсказания класса объекта. Есть различные способы посчитать его важность. Один из простых способов сделать это для Random Forest - посчитать out-of-bag ошибку предсказания `err_oob`, а затем перемешать значения признака `j` и посчитать ее (`err_oob_j`) еще раз. Оценкой важности признака `j` для одного дерева будет разность `err_oob_j - err_oob`, важность для всего леса считается как среднее значение важности по деревьям.

Реализуйте функцию `feature_importance`, которая принимает на вход Random Forest и возвращает массив, в котором содержится важность для каждого признака.

In [7]:
def feature_importance(rfc):
    def classification_error(tree, X_oob, y_oob, feature):
        saved_column = deepcopy(X_oob[:, feature])
        np.random.shuffle(X_oob[:, feature])
        res = 1 - np.mean(tree.predict(X_oob) == y_oob)
        X_oob[:, feature] = saved_column
        return res
    
    n_features = rfc.oobs[0][0].shape[1]
    
    features_importance = []
    for feature in range(n_features):
        err_oob = np.array([1 - np.mean(tree.predict(X_oob) == y_oob) for tree, (X_oob, y_oob) in zip(rfc.trees, rfc.oobs)])
        err_oob_j = np.array([classification_error(tree, X_oob, y_oob, feature) for tree, (X_oob, y_oob) in zip(rfc.trees, rfc.oobs)])
        features_importance.append(np.mean(err_oob_j - err_oob))
        
    return features_importance
            

def most_important_features(importance, names, k=20):
    # Выводит названия k самых важных признаков
    indices = np.argsort(importance)[::-1][:k]
    return np.array(names)[indices]

Наконец, пришло время протестировать наше дерево на простом синтетическом наборе данных. В результате точность должна быть примерно равна `1.0`, наибольшее значение важности должно быть у признака с индексом `4`, признаки с индексами `2` и `3`  должны быть одинаково важны, а остальные признаки - не важны совсем.

In [8]:
def synthetic_dataset(size):
    X = [(np.random.randint(0, 2), np.random.randint(0, 2), i % 6 == 3, 
          i % 6 == 0, i % 3 == 2, np.random.randint(0, 2)) for i in range(size)]
    y = [i % 3 for i in range(size)]
    return np.array(X), np.array(y)

X, y = synthetic_dataset(1000)
rfc = RandomForestClassifier(n_estimators=10)
rfc.fit(X, y)
print("Accuracy:", np.mean(rfc.predict(X) == y))
print("Importance:", feature_importance(rfc))

Accuracy: 1.0
Importance: [0.001033284984478633, 0.003551823147277666, 0.1631108961455771, 0.14189986993677386, 0.38650441693428295, 0.005812445834890834]


### Задание 4 (1 балл)
Теперь поработаем с реальными данными.

Выборка состоит из публичных анонимизированных данных пользователей социальной сети Вконтакте. Первые два столбца отражают возрастную группу (`zoomer`, `doomer` и `boomer`) и пол (`female`, `male`). Все остальные столбцы являются бинарными признаками, каждый из них определяет, подписан ли пользователь на определенную группу/публичную страницу или нет.\
\
Необходимо обучить два классификатора, один из которых определяет возрастную группу, а второй - пол.\
\
Эксперименты с множеством используемых признаков и подбор гиперпараметров приветствуются. Лес должен строиться за какое-то разумное время.

In [9]:
def read_dataset(path):
    dataframe = pandas.read_csv(path, header=0)
    dataset = dataframe.values.tolist()
    random.shuffle(dataset)
    y_age = [row[0] for row in dataset]
    y_sex = [row[1] for row in dataset]
    X = [row[2:] for row in dataset]
    
    return np.array(X), np.array(y_age), np.array(y_sex), list(dataframe.columns)[2:]

In [10]:
X, y_age, y_sex, features = read_dataset("vk.csv")
X_train, X_test, y_age_train, y_age_test, y_sex_train, y_sex_test = train_test_split(X, y_age, y_sex, train_size=0.9)

#### Возраст

In [11]:
rfc = RandomForestClassifier(max_depth=6, min_samples_leaf=100, n_estimators=30)

rfc.fit(X_train, y_age_train)
print("Accuracy:", np.mean(rfc.predict(X_test) == y_age_test))
print("Most important features:")
for i, name in enumerate(most_important_features(feature_importance(rfc), features, 20)):
    print(str(i+1) + ".", name)

Accuracy: 0.6582597730138714
Most important features:
1. 4ch
2. ovsyanochan
3. styd.pozor
4. rhymes
5. mudakoff
6. dayvinchik
7. pixel_stickers
8. reflexia_our_feelings
9. i_d_t
10. webestano
11. pustota_diary
12. pozor
13. iwantyou
14. pravdashowtop
15. rapnewrap
16. soverwenstvo.decora
17. tumblr_vacuum
18. flatme
19. webmland
20. ne1party


#### Пол

In [12]:
rfc = RandomForestClassifier(max_depth=5, n_estimators=30)
rfc.fit(X_train, y_sex_train)
print("Accuracy:", np.mean(rfc.predict(X_test) == y_sex_test))
print("Most important features:")
for i, name in enumerate(most_important_features(feature_importance(rfc), features, 20)):
    print(str(i+1) + ".", name)

Accuracy: 0.798234552332913
Most important features:
1. mudakoff
2. rapnewrap
3. 40kg
4. modnailru
5. 9o_6o_9o
6. 4ch
7. recipes40kg
8. cook_good
9. girlmeme
10. be.beauty
11. bot_maxim
12. academyofman
13. be.women
14. igm
15. beauty
16. zerofat
17. sh.cook
18. bon
19. thesmolny
20. combovine


### CatBoost
В качестве аьтернативы попробуем CatBoost. 

Устаниовить его можно просто с помощью `pip install catboost`. Туториалы можно найти, например, [здесь](https://catboost.ai/docs/concepts/python-usages-examples.html#multiclassification) и [здесь](https://github.com/catboost/tutorials/blob/master/python_tutorial.ipynb). Главное - не забудьте использовать `loss_function='MultiClass'`.\
\
Сначала протестируйте CatBoost на синтетических данных. Выведите точность и важность признаков.

In [13]:
from catboost import CatBoostClassifier

In [14]:
X, y = synthetic_dataset(1000)
model = CatBoostClassifier(iterations=10,
                           learning_rate=1,
                           depth=5,
                           loss_function='MultiClass')
model.fit(X, y)
print("Accuracy:", np.mean(model.predict(X).flatten() == y))
print("Importance:", model.get_feature_importance())

0:	learn: 0.1149784	total: 46.9ms	remaining: 422ms
1:	learn: 0.0423112	total: 48ms	remaining: 192ms
2:	learn: 0.0202855	total: 49.1ms	remaining: 114ms
3:	learn: 0.0115630	total: 50.2ms	remaining: 75.2ms
4:	learn: 0.0063923	total: 51.1ms	remaining: 51.1ms
5:	learn: 0.0040969	total: 51.9ms	remaining: 34.6ms
6:	learn: 0.0029190	total: 52.7ms	remaining: 22.6ms
7:	learn: 0.0025008	total: 53.8ms	remaining: 13.4ms
8:	learn: 0.0021836	total: 54.9ms	remaining: 6.1ms
9:	learn: 0.0017622	total: 55.7ms	remaining: 0us
Accuracy: 1.0
Importance: [8.35431637e-04 2.14771099e-03 2.79299620e+01 2.78741308e+01
 4.41909557e+01 1.96830410e-03]


### Задание 5 (3 балла)
Попробуем применить один из используемых на практике алгоритмов. В этом нам поможет CatBoost. Также, как и реализованный ними RandomForest, применим его для определения пола и возраста пользователей сети Вконтакте, выведите названия наиболее важных признаков так же, как в задании 3.\
\
Эксперименты с множеством используемых признаков и подбор гиперпараметров приветствуются.

In [15]:
X, y_age, y_sex, features = read_dataset("vk.csv")
X_train, X_test, y_age_train, y_age_test, y_sex_train, y_sex_test = train_test_split(X, y_age, y_sex, train_size=0.9)
X_train, X_eval, y_age_train, y_age_eval, y_sex_train, y_sex_eval = train_test_split(X_train, y_age_train, y_sex_train, train_size=0.8)

#### Возраст

In [16]:
model = CatBoostClassifier(iterations=300,
                           learning_rate=1,
                           depth=4,
                           verbose=0,
                           loss_function='MultiClass')
model.fit(X_train, y_age_train)
print("Accuracy:", np.mean(model.predict(X_test).flatten() == y_age_test))
print("Most important features:")
for i, name in enumerate(most_important_features(model.get_feature_importance(), features, 10)):
    print(str(i+1) + ".", name)

Accuracy: 0.725094577553594
Most important features:
1. mudakoff
2. ovsyanochan
3. leprum
4. 4ch
5. styd.pozor
6. fuck_humor
7. xfilm
8. rhymes
9. tumblr_vacuum
10. kino_mania


#### Пол

In [17]:
model = CatBoostClassifier(iterations=300,
                           learning_rate=1,
                           depth=4,
                           verbose=0,
                           loss_function='MultiClass')
model.fit(X_train, y_sex_train)
print("Accuracy:", np.mean(model.predict(X_test).flatten() == y_sex_test))
print("Most important features:")
for i, name in enumerate(most_important_features(model.get_feature_importance(), features, 10)):
    print(str(i+1) + ".", name)

Accuracy: 0.8575031525851198
Most important features:
1. 40kg
2. mudakoff
3. igm
4. girlmeme
5. modnailru
6. academyofman
7. femalemem
8. cook_good
9. zerofat
10. be.beauty
